In [18]:
from IPython.core.display import display, HTML
from h5py import File
import pandas as pd
import numpy as np
from pathlib import Path
import os
root_path = Path(r"C:\Users\robin\Documents\Cours\Poly\Recherche\FIBI_Recherche")
print(root_path)
os.chdir(str(root_path))
from FIBI.data.analyse_cpp_results.clustering.extract_infos import parse_key, parse_metadata, DataCategory, corresp_impr_class
import json
import re

C:\Users\robin\Documents\Cours\Poly\Recherche\FIBI_Recherche


C:\Users\robin\AppData\Local\Temp\ipykernel_12940\232573296.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


## Load data

In [4]:
data = []
with open("data/mapping_tsplib.json","r") as f:
    l_mapping = json.load(f)
with File("data/table2_exp_clust.hdf5", "r") as f:
    for k,v in f["metadata"].items():
        dico_key = parse_key(k)
        v = np.copy(v)
        dico_value = {"key_dataset":k,**parse_metadata(v)}
        dico_value["SEED_POINTS"] = l_mapping[dico_value["SEED_POINTS"]]
        data.append(dico_value)
df = pd.DataFrame(data)


In [3]:
display(df.head())

,key_dataset,NUM_POINTS,NUM_DIM,POINTS_DISTR,NUM_CLUST,CLUST_IMPR,IMPR_CLASS,IT_ORDER,SEED_POINTS,SEED_ASSIGN,init_cost,final_cost,num_iter,num_iter_glob,duration
0,"0,52,3,0,0,2,0,1",52,2,3,2,random,BICL,CURR,0,0,1.0,0.0,0,11276757,5.889693
1,"0,52,3,1,0,2,0,1",52,2,3,2,random,BICL,CURR,0,1,1.0,0.0,1,11256356,5.889693
2,"0,52,3,10,1,2,0,1",52,2,3,2,random,FICL,CURR,0,10,1.0,0.0,10,11349613,5.889693
3,"0,52,3,11,1,2,0,1",52,2,3,2,random,FICL,CURR,0,11,1.0,0.0,11,11227938,5.889693
4,"0,52,3,12,1,2,0,1",52,2,3,2,random,FICL,CURR,0,12,1.0,0.0,12,11251089,5.889693


## Verify that we have all of the required results

In [6]:
def check_print(cond:bool):
    if cond:
        print("✅")
    else:
        print("❌")

Datasets

Expected: (listed in mapping_tsplib.json)

Found:

In [7]:
expected = l_mapping
seed_points = list(df[str(DataCategory.SEED_POINTS)].unique())
seed_points.sort()
print(seed_points)
check_print(set(seed_points) == set(expected))

['berlin52', 'ch130', 'ch150', 'd1291', 'd198', 'eil51', 'kroC100', 'kroD100', 'lin105', 'linhp318', 'p654', 'pcb1173', 'pcb442', 'pr1002', 'pr2392', 'pr299', 'pr439', 'rd400', 'rl1323', 'tsp225', 'u1060', 'u1432', 'u574']
✅


Number of different point and assign seed per number of point x algorithm

Expected:

- for n in [20, 30, ..., 150] : 1000
- for n in [200, 250, ..., 500] : 100
- for n in [500, 600, ..., 1000] : 100

Found:

In [31]:
groups_counts = df.groupby([str(DataCategory.SEED_POINTS),str(DataCategory.IMPR_CLASS),str(DataCategory.NUM_CLUST)]).count().reset_index()
expected = 10
cond = True

for data in l_mapping:
    for impr_class in corresp_impr_class.values():
        for num_clust in [2,4,8,16,32,64,128,256]:
            query = f"{str(DataCategory.SEED_POINTS)} == '{data}' and {str(DataCategory.IMPR_CLASS)} == '{impr_class}' and {str(DataCategory.NUM_CLUST)} == {num_clust}"
            df_tmp = groups_counts.query(query)
            if len(df_tmp) == 0:
                match = re.search(r"\d+",data)
                if int(match.group()) > num_clust:
                    cond = False
                    print(f"{data} {impr_class} {num_clust}",end=" | ")
                continue
            cond = cond and df_tmp.iloc[0][str(DataCategory.SEED_ASSIGN)] == expected
check_print(cond)

✅


## Statistics

### Computed statistics

In [36]:
df_table1_src = df[[
    str(DataCategory.SEED_POINTS),
    str(DataCategory.NUM_CLUST),
    str(DataCategory.IMPR_CLASS),
    str(DataCategory.final_cost),
    str(DataCategory.num_iter),
    str(DataCategory.num_iter_glob),
    str(DataCategory.duration),
    ]]
df_table1 = df_table1_src.copy().groupby([str(DataCategory.SEED_POINTS),str(DataCategory.NUM_CLUST),str(DataCategory.IMPR_CLASS)]).mean().reset_index()
df_table1 = df_table1.query(f"{str(DataCategory.IMPR_CLASS)} != 'BICL'")
df_table1.sort_values(by=[str(DataCategory.SEED_POINTS),str(DataCategory.NUM_CLUST),str(DataCategory.IMPR_CLASS)][::-1], inplace=True)
df_table1.reset_index(drop=True, inplace=True)
df_table1_str = df_table1.to_html(formatters={
    str(DataCategory.SEED_POINTS): '{:s}'.format,
    str(DataCategory.NUM_CLUST): '{:d}'.format,
    str(DataCategory.IMPR_CLASS): '{:s}'.format,
    str(DataCategory.final_cost): '{:.2f}'.format,
    str(DataCategory.num_iter): '{:.0f}'.format,
    str(DataCategory.num_iter_glob): '{:.0f}'.format,
    str(DataCategory.duration): '{:.4e}'.format,
})
display(HTML(df_table1_str))

,SEED_POINTS,NUM_CLUST,IMPR_CLASS,final_cost,num_iter,num_iter_glob,duration
0,berlin52,2,FICL,0.00,14,11228368,5.8897e+00
1,ch130,2,FICL,1.00,174,9922587,5.7551e+00
2,ch150,2,FICL,2.00,334,11628868,6.7721e+00
3,d1291,2,FICL,3.00,494,1451246324,8.5970e+02
4,d198,2,FICL,4.00,654,154749215,4.5659e+01
5,eil51,2,FICL,5.00,814,31417,1.9799e-02
6,kroC100,2,FICL,6.00,974,182567880,6.3787e+01
7,kroD100,2,FICL,7.00,1134,167316047,6.8042e+01
8,lin105,2,FICL,8.00,1294,94119831,3.3940e+01
9,linhp318,2,FICL,9.00,1454,667951534,3.5263e+02


## Export results for visualization

In [37]:
computed = df_table1_src.copy()

computed = computed[["SEED_POINTS","NUM_CLUST","IMPR_CLASS","final_cost","num_iter","duration"]]
computed["source"] = "computed"
new_df = computed
display(new_df)

,SEED_POINTS,NUM_CLUST,IMPR_CLASS,final_cost,num_iter,duration,source
0,berlin52,2,BICL,0.0,0,5.889693,computed
1,berlin52,2,BICL,0.0,1,5.889693,computed
2,berlin52,2,FICL,0.0,10,5.889693,computed
3,berlin52,2,FICL,0.0,11,5.889693,computed
4,berlin52,2,FICL,0.0,12,5.889693,computed
...,...,...,...,...,...,...,...
3355,linhp318,256,FICL,9.0,1595,0.117895,computed
3356,linhp318,256,FICL,9.0,1596,0.138277,computed
3357,linhp318,256,FICL,9.0,1597,0.126451,computed
3358,linhp318,256,FICL,9.0,1598,0.142994,computed


In [40]:
def q1(x):
    return x.quantile(0.25)

def median(x):
    return x.quantile(0.50)
def q3(x):
    return x.quantile(0.75)
f = ["mean","std","min",q1,median,q3,"max"]
df_avg = new_df.groupby(["SEED_POINTS","IMPR_CLASS","source"]).agg(f)
df_avg.reset_index(inplace=True)
df_avg.fillna(-1,inplace=True)
Lout = []
for elem in df_avg.to_dict(orient="records"):
    dico = {}
    for k,v in elem.items():
        if k[1] == '':
            dico[k[0]] = v
        else:
            if k[0] not in dico:
                dico[k[0]] = {}
            dico[k[0]][k[1]] = v
    Lout.append(dico)
dico_filtered = {}
for n in new_df["SEED_POINTS"].unique():
    for nClust in new_df["NUM_CLUST"].unique():
        for impr_class in new_df["IMPR_CLASS"].unique():
            tmp_dico = new_df.query(f"SEED_POINTS == '{n}' and IMPR_CLASS == '{impr_class}' and source == 'computed'").to_dict(orient="records")
            if len(tmp_dico) == 0:
                continue
            tmp_dico["INIT_CHOICE"] = "random"
            dico_filtered[f"{n}_{impr_class}_computed"] = tmp_dico
            
dico_out = {"grouped":Lout,"filtered":dico_filtered}

with open("data/table2_clust_visualize.json","w") as f:
    json.dump(dico_out,f,indent=2)